# **JSWMM - Fossolo example**
Component:
- PreRunoff
- Runoff
- FlowRateDispatcher
- Routing

Complete documentation: https://osf.io/349vd/

### **Inputs description**
#### **General description**
The input of the model are the file <em>.inp</em> generated by EPA SWMM. The topology of the network and the sim files.
The file <em>.inp</em> collects all the network assigned properties. An example of how to generate this file is described in https://growworkinghard.altervista.org/draft-of-sewer-design/.
The topology file describes how the nodes are connected. Each node is a model solution. See the example for a better explanation.


### **PRERUNOFF COMPONENT**
#### *Inputs*
@In <b>areaName</b>: name of the area where the runoff must be evaluated;

@In <b>aLPP</b>: constant coefficient of the power law (see Output) - defined in Hello.sim;

@In <b>nLPP</b>: constant coefficient of the power law (see Output) - defined in Hello.sim;

@In <b>numberOfCurves</b>: number of curves used to evaluate the maximum (in this version must be in the range [1-3]) - defined in Hello.sim.

#### *Outputs*
@Out <b>adaptedRainfallData</b>: contains the behaviour of rainfall data. It is evaluated at 3, 5 and 10 minutes using the power law:

$$I = a\cdot t_p^{(n-1)}$$

<em>a</em> and <em>n</em> are inputs of the model. <em>$t_p$</em> is the rainfall period.

The output is an HashMap<Integer, LinkedHashMap<Instant, Double>>  where the HM key is the curve (3,5 and 10 minutes). The inner LHM contains the values of rainfall over time.

### **RUNOFF COMPONENT**
#### *Inputs*
@In <b>areaName</b>: name of the area where the runoff must be evaluated;

@In <b>nodeName</b>: name of the node where the runoff is concentrated.

#### *Outputs*
@OutNode <b>runoffFlowRate</b>: contains the response of the area to the input of rainfall. It is in the same form of the adaptedRainfallData. It is an HashMap<Integer, LinkedHashMap<Instant, Double>>  where the HM key is the curve (3,5 and 10 minutes). The inner LHM contains the values of flowrate over time.

### **FLOWRATEDISPATCHER COMPONENT**
#### *Inputs*
@InNode <b>flowRate</b>: a series of HashMap<Integer, LinkedHashMap<Instant, Double>> that bring to next node the flowrate due to the upper part of the network.

@InNode <b>dataStructure</b>: full data structure, updated to the last upper node.

#### *Outputs*
@OutNode <b>dataStructure</b>: full data structure, updated to the current upper node. In this the flow rate at the current junction is the sum of all contributes above.

### **ROUTING COMPONENT**
#### *Inputs*
@InNode <b>dataStructure</b>: full data structure, updated to the last upper node.

#### *Outputs*
@OutNode <b>routingFlowRate</b>: an HashMap<Integer, LinkedHashMap<Instant, Double>> that bring to next NET3 node the flowrate due to the upper part of the network, routed to lower junction of the conduit.

### **Outputs description**


In [28]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from IPython.display import Image
import os

In [29]:
###########################
### SET THE FOLDER PATH ###
###########################
projFolder = '/home/daniele/git/github/GEOframeOMSProjects/OMS_Project_JSWMM/'

In [30]:
os.chdir(projFolder)

In [31]:
os.listdir()

['src',
 'build.xml',
 'build',
 'output',
 '.gitignore',
 '.oms',
 'dist',
 '.git',
 'lib',
 'data',
 'simulation',
 'docs',
 'LICENSE']

In [32]:
f=open("./data/fossolo.inp", "r")
print(f.read())

[TITLE]
;;Project Title/Notes

[OPTIONS]
;;Option             Value
FLOW_UNITS           CMS
INFILTRATION         HORTON
FLOW_ROUTING         STEADY
LINK_OFFSETS         DEPTH
MIN_SLOPE            0
ALLOW_PONDING        NO
SKIP_STEADY_STATE    NO

START_DATE           01/01/2018
START_TIME           00:00:00
REPORT_START_DATE    01/01/2018
REPORT_START_TIME    00:00:00
END_DATE             01/01/2018
END_TIME             01:00:00
SWEEP_START          01/01
SWEEP_END            12/31
DRY_DAYS             0
REPORT_STEP          00:00:30
WET_STEP             00:01:00
DRY_STEP             01:00:00
ROUTING_STEP         0:00:30 

INERTIAL_DAMPING     PARTIAL
NORMAL_FLOW_LIMITED  BOTH
FORCE_MAIN_EQUATION  H-W
VARIABLE_STEP        0.75
LENGTHENING_STEP     0
MIN_SURFAREA         1.14
MAX_TRIALS           8
HEAD_TOLERANCE       0.0015
SYS_FLOW_TOL         5
LAT_FLOW_TOL         5
MINIMUM_STEP         0.5
THREADS              1

[EVAPORATION]
;;Data Source    Parameters
;;-------------- --------

In [33]:
f=open("./data/topo.csv", "r")
print(f.read())

1 11 
2 12
3 13
4 14
5 15
6 16
7 17
8 18
9 19
10 20
11 13
12 13
14 16
15 16
13 17
16 17
17 20
18 20
19 20
20 0



In [34]:
f=open("./simulation/Hello.sim", "r")
print(f.read())

import static oms3.SimBuilder.instance as OMS3
import com.github.geoframecomponents.jswmm.dataStructure.SWMMobject

OMS3.graph(path: "./data/topo.csv",
           simpath: "./simulation/") {
           	
	resource "$oms_prj/lib"
	
	//build()

    graph(traverser: "downstream.all") {
    	parameter {
            "aLPP" 60.4
            "nLPP" 0.61
            "numberOfCurves" 3
        }
    }
    
    flags {
    	"1" "{overwrite}"
    	"2" "{overwrite}"
    	"3" "{overwrite}"
    	"4" "{overwrite}"
    	"5" "{overwrite}"
    	"6" "{overwrite}"
    	"7" "{overwrite}"
    	"8" "{overwrite}"
    	"9" "{overwrite}"
    	"10" "{overwrite}"
    }
    
    model() {
    	   	
        components {
            "c" "ex0.OutT"
        }
        
        parameter {
            "c.datastructure" new SWMMobject("${oms_prj}/data/fossolo.inp", 3)
        }
        
        outFluxes {
            "c.datastructure" ""
        }
    } 
}


In [35]:
!docker run --rm -it -v $(pwd):/work omslab/oms:beta simulation/Hello.sim

build/ folder exists: project already built.
index: 1 - calibrate: false - overwrite: true
index: 2 - calibrate: false - overwrite: true
index: 3 - calibrate: false - overwrite: true
index: 4 - calibrate: false - overwrite: true
index: 5 - calibrate: false - overwrite: true
index: 6 - calibrate: false - overwrite: true
index: 7 - calibrate: false - overwrite: true
index: 8 - calibrate: false - overwrite: true
index: 9 - calibrate: false - overwrite: true
index: 10 - calibrate: false - overwrite: true
log4j:WARN No appenders could be found for logger (org.apache.commons.beanutils.converters.BooleanConverter).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
Routing on 12
Q_MAX 0.492
D 0.7
Routing on 11
Q_MAX 0.535
D 0.7
Routing on 15
Q_MAX 0.576
D 0.8
Routing on 14
Q_MAX 0.212
D 0.5
Routing on 13
Q_MAX 1.168
D 0.7
Routing on 18
Q_MAX 0.329
D 0.4
Routing on 16
Q_MAX 0.868
D 1.0
Routing on 17
Q_MAX 

**Read the JSWMM output**

In [36]:
##Work in progress